# GitHub上でCodex CLIを使用してCIの失敗を自動修正する

## このクックブックの目的

このクックブックでは、OpenAI Codex CLIをCI/CDパイプラインに組み込んで、ビルドやテストが失敗した際にcodexが自動的に修正案を生成・提案する方法を説明します。以下は、GitHub ActionsでCIを実行するnodeプロジェクトでの例です。

## エンドツーエンドフロー

以下が実装するパイプラインフローです：

<img src="../../images/ci-codex-workflow.png"  width="700"/>

## 前提条件

- Actionsワークフローを持つGitHubリポジトリ

- GitHubの設定で https://github.com/{org-name}/{repo-name}/settings/secrets/actions にて `OPENAI_API_KEY` を環境変数として作成する必要があります。これは組織レベルでも設定可能です（複数のリポジトリ間でシークレットを共有するため）

- Codexは `codex login` を使用するためにpythonが前提条件として必要です

- リポジトリでアクションがPRを作成できるようにする設定を確認する必要があり、組織でも同様の設定が必要です：


<img src="../../images/github-pr-settings.png" width="700"/>

## ステップ1: GitHub ActionをCIパイプラインに追加する

以下のYAMLは、CIが失敗した際に自動的にトリガーされ、Codexをインストールし、codex execを使用して、失敗したブランチに修正内容でPRを作成するGitHub Actionを示しています。「CI」を監視したいワークフローの名前に置き換えてください。

```yaml
name: Codex Auto-Fix on Failure

on:
  workflow_run:
    # プライマリCIワークフローの実行完了後にこのジョブをトリガー
    workflows: ["CI"]
    types: [completed]

permissions:
  contents: write
  pull-requests: write

jobs:
  auto-fix:
    # 参照されたワークフローが失敗で終了した場合のみ実行
    if: ${{ github.event.workflow_run.conclusion == 'failure' }}
    runs-on: ubuntu-latest
    env:
      OPENAI_API_KEY: ${{ secrets.OPENAI_API_KEY }}
      FAILED_WORKFLOW_NAME: ${{ github.event.workflow_run.name }}
      FAILED_RUN_URL: ${{ github.event.workflow_run.html_url }}
      FAILED_HEAD_BRANCH: ${{ github.event.workflow_run.head_branch }}
      FAILED_HEAD_SHA: ${{ github.event.workflow_run.head_sha }}
    steps:
      - name: OpenAI APIキーの設定確認
        run: |
          if [ -z "$OPENAI_API_KEY" ]; then
            echo "OPENAI_API_KEYシークレットが設定されていません。自動修正をスキップします。" >&2
            exit 1
          fi
      - name: 失敗したリファレンスをチェックアウト
        uses: actions/checkout@v4
        with:
          ref: ${{ env.FAILED_HEAD_SHA }}
          fetch-depth: 0

      - name: Node.jsのセットアップ
        uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'

      - name: 依存関係のインストール
        run: |
          if [ -f package-lock.json ]; then npm ci; else npm i; fi
      - name: Codexの実行
        uses: openai/codex-action@main
        id: codex
        with:
          openai_api_key: ${{ secrets.OPENAI_API_KEY }}
          prompt: "あなたはJestテストとGitHub Actionsを使用するNode.jsモノレポで作業しています。リポジトリを読み込み、テストスイートを実行し、すべてのテストを通すために必要な最小限の変更を特定し、その変更のみを実装して停止してください。関連のないコードやファイルをリファクタリングしないでください。変更は小さく外科的に行ってください。"
          codex_args: '["--config","sandbox_mode=\"workspace-write\""]'

      - name: テストの検証
        run: npm test --silent

      - name: 修正内容でプルリクエストを作成
        if: success()
        uses: peter-evans/create-pull-request@v6
        with:
          commit-message: "fix(ci): Codexによる失敗テストの自動修正"
          branch: codex/auto-fix-${{ github.event.workflow_run.run_id }}
          base: ${{ env.FAILED_HEAD_BRANCH }}
          title: "CodexによるCI失敗の自動修正"
          body: |
            Codexがワークフロー `${{ env.FAILED_WORKFLOW_NAME }}` のCI失敗に対応してこのPRを自動生成しました。
            失敗した実行: ${{ env.FAILED_RUN_URL }}
            ヘッドブランチ: `${{ env.FAILED_HEAD_BRANCH }}`
            このPRにはCIを通すことのみを目的とした最小限の変更が含まれています。
```

## ステップ2: Actionsワークフローの開始

Repoの下にあるActionsタブに移動して、Actionsワークフロー内の失敗したジョブを確認できます。

<img src="../../images/failing-workflow.png" width="700"/>

失敗したワークフローの完了時に、Codexワークフローがトリガーされる必要があります。


<img src="../../images/codex-workflow.png" width="700"/>

## ステップ3: CodexがレビューのためのPRを作成したことを確認する
Codexワークフローの実行が完了すると、フィーチャーブランチ`codex/auto-fix`からプルリクエストが開かれるはずです。すべてが正常に見えるかを確認し、その後マージしてください。

<img src="../../images/codex-pr.png" width="700"/>

## 結論

この自動化により、OpenAI Codex CLIとGitHub Actionsがシームレスに統合され、失敗したCIの実行に対して自動的に修正案を提案できます。

Codexを活用することで、手動での介入を減らし、コードレビューを加速し、mainブランチを健全な状態に保つことができます。このワークフローにより、テストの失敗が迅速かつ効率的に対処され、開発者はより価値の高いタスクに集中できるようになります。codex-cliとその機能について詳しくは[こちら](https://github.com/openai/codex/)をご覧ください。